In [334]:
import numpy as np
import datasets
from scipy.optimize import minimize
from scipy.special import expit
from sklearn.metrics import log_loss
from softmax import softmax, log_softmax
import theano
from theano import tensor as T
import tensorflow as tf
from scipy import stats
from numpy.linalg import inv, eigh
from random import sample
from scipy.linalg import sqrtm

In [344]:
reload(datasets)
datasets.spam()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.000,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.000,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.000,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.000,0.135,0.000,0.000,3.537,40,191,1
5,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.000,0.223,0.000,0.000,0.000,0.000,3.000,15,54,1
6,0.00,0.00,0.00,0.0,1.92,0.00,0.00,0.00,0.00,0.64,...,0.000,0.054,0.000,0.164,0.054,0.000,1.671,4,112,1
7,0.00,0.00,0.00,0.0,1.88,0.00,0.00,1.88,0.00,0.00,...,0.000,0.206,0.000,0.000,0.000,0.000,2.450,11,49,1
8,0.15,0.00,0.46,0.0,0.61,0.00,0.30,0.00,0.92,0.76,...,0.000,0.271,0.000,0.181,0.203,0.022,9.744,445,1257,1
9,0.06,0.12,0.77,0.0,0.19,0.32,0.38,0.00,0.06,0.00,...,0.040,0.030,0.000,0.244,0.081,0.000,1.729,43,749,1


In [335]:
# X, y = datasets.htwt()
X, y = datasets.iris()
Y = datasets.one_hot(y)
N, D = X.shape
N, C = Y.shape
Xaug = np.hstack([np.ones((N, 1)), X])

In [213]:
z = T.dvector()
decode = lambda params: (params[1:].reshape((D, 1)), params[0])
mu = lambda w, b: T.nnet.sigmoid(T.dot(X, w) + b)
loss = T.sum(T.nnet.binary_crossentropy(mu(*decode(z)), y.reshape((N, 1))))
grad = theano.function([z], T.grad(loss, z))
hess = theano.function([z], T.hessian(loss, z))
loss = theano.function([z], loss)

In [215]:
mu = lambda w: expit(Xaug.dot(w))
loss = lambda w: log_loss(y, mu(w), normalize = False)
grad = lambda w: Xaug.T.dot(mu(w) - y)
S = lambda mu: np.diag(mu * (1 - mu))
hess = lambda w: Xaug.T.dot(S(mu(w))).dot(Xaug)

In [216]:
w = np.array([0.0] * (D + 1))
w = minimize(loss, w).x
w = minimize(loss, w, method = 'Newton-CG', jac = grad, hess = hess, tol = 1e-6).x
yhat = expit(Xaug.dot(w)) > 0.5
print '%0.3f' % np.mean(yhat != y)

0.119


In [6]:
decode = lambda params: params.reshape((D + 1, C))
z = T.dvector()
mu = lambda W: T.nnet.softmax(T.dot(Xaug, W))
loss = T.sum(T.nnet.categorical_crossentropy(mu(decode(z)), Y))
grad = theano.function([z], T.grad(loss, z))
hess = theano.function([z], T.hessian(loss, z))    
loss = theano.function([z], loss)

In [7]:
decode = lambda params: np.array(params).reshape((D + 1, C))

mu = lambda W: softmax(Xaug.dot(W))
loss = lambda params: log_loss(y, mu(decode(params)), normalize = False)

def grad(params):
    Z = mu(decode(params)) - Y
    return sum([np.kron(Xaug[i], z) for i, z in enumerate(Z)])

def hess(params):
    Mu = mu(decode(params))
    o = lambda x: np.outer(x, x)
    return sum([np.kron(o(Xaug[i]), np.diag(z) - o(z)) for i, z in enumerate(Mu)])

In [8]:
params = [0] * (D + 1) * C
params = minimize(loss, params, method = 'Newton-CG', jac = grad, hess = hess, tol = 1e-6).x
W = params.reshape((D + 1, C))
yhat = np.argmax(softmax(Xaug.dot(W)), axis = 1)
print '%0.3f' % np.mean(yhat != y)

0.114


In [108]:
# bayesian logistic regression
mvn = stats.multivariate_normal
log_prior = lambda w: mvn.logpdf(w[1:], np.zeros(D), 100 * np.eye(D))
mu = lambda w: expit(Xaug.dot(w))
nll = lambda w: log_loss(y, mu(w), normalize = False)
E = lambda w: nll(w) - log_prior(w)
mN = minimize(E, np.zeros(D + 1)).x
S = lambda mu: np.diag(mu * (1 - mu))
H = Xaug.T.dot(S(mu(mN))).dot(Xaug) + 1.0 / 200
VN = inv(H)

In [119]:
# monte carlo approximation
posterior = mvn(mN, VN)
yhat = np.mean([mu(w) for w in posterior.rvs(1000)], axis = 0) > 0.5
print '%0.3f' % np.mean(yhat != y)

0.119


In [110]:
# probit approximation
yhat = (1 + np.pi * Xaug.dot(VN).dot(Xaug.T) / 8) ** -0.5 * Xaug.dot(mN) > 0.5
print '%0.3f' % np.mean(yhat != y)

0.110


In [299]:
# stochastic gradient descent
w = np.array([0.0] * (D + 1))
s = w.copy()
tau0 = 1e-6
eta = 1e-2
batch_size = 100
epochs = 1000

for _ in xrange(epochs):
    i = sample(range(N), batch_size)
    mu = expit(Xaug[i].dot(w))
    g = Xaug[i].T.dot(mu - y[i])
    s += g ** 2
    w -= eta * g / (tau0 + np.sqrt(s)) # adagrad

yhat = expit(Xaug.dot(w)) > 0.5
print '%0.3f' % np.mean(yhat != y)

0.219


In [338]:
# fisher's linear discriminant algorithm
mus = []

L = 2
aaT = lambda a: np.outer(a, a)
aTa = lambda a: np.array(a).T.dot(a)
mu = np.mean(X)
SW = np.zeros((D, D))
SB = np.zeros((D, D))

for c in xrange(C):
    Xc = X[y == c]
    muc = np.mean(Xc, axis = 0) * 1.0
    mus.append(muc)
    SW += sum([aaT(xc - muc) for xc in Xc])
    SB += len(Xc) * aaT(muc - mu)

if C == 2:
    W = inv(SW).dot(mus[1] - mus[0])
else:
    SW_inv_root = sqrtm(inv(SW))
    _, v = eigh(SW_inv_root.dot(SB).dot(SW_inv_root))
    U = v[:, : -L - 1 : -1]
    W = SW_inv_root.dot(U)
    
Z = X.dot(W)
d = np.array([[aTa(z - mc) for z in Z] for mc in np.dot(mus, W)]).T
yhat = np.argmin(d, axis = 1) 
print '%0.3f' % np.mean(yhat != y)

0.020
